In [1]:
# Set Up
import os
import torch
from torch import nn

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision

import matplotlib.pyplot as plt
import numpy as np

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Mounted at /gdrive
/gdrive


In [2]:
# Data Loader
class DataLoader():
  def __init__(self, dir="MyDrive/data", width=300, height=432):
    self.width = width
    self.height = height
    self.root = dir

  def load(self, dataset="/valid", batchSize=10, shuffle=True, workers=2):
    transform = transforms.Compose([transforms.Resize((124, 124)),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    dataset = datasets.ImageFolder(root=self.root + dataset, transform=transform)
    dataset = torch.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=shuffle, num_workers=workers)
    return dataset


In [3]:
# Multilayer Perceptron
class MLP(nn.Module):
  def __init__(self, inputSize=124*124*3, outputSize=8, lr=0.01):
    super().__init__()

    self.net = nn.Sequential(
      nn.flatten(),
      nn.Linear(inputSize, 1024),
      nn.ReLU(),
      nn.Linear(1024, 512),
      nn.ReLU(),
      nn.Linear(512, 256),
      nn.ReLU(),
      nn.Linear(256, 124),
      nn.ReLU(),
      nn.Linear(124, outputSize),
      nn.ReLU(),
    )

    self.lr = lr

  def forward(self, X):
    return self.net(X)

  def loss(self, y_hat, y):
    # print(y, y_hat)
    fn = nn.CrossEntropyLoss()
    return fn(y_hat, y)

  def configureOptimiser(self):
    return torch.optim.Adam(self.parameters(), self.lr)

In [4]:
# Trainer Class
class Trainer():
  def __init__(self, nEpochs=3):
    self.maxEpochs = nEpochs

  # The fitting step
  def fit(self, model, data):

    self.data = data

    # configure the optimizer
    self.optimiser = model.configureOptimiser()
    self.model = model

    for epoch in range(self.maxEpochs):
      self.fitEpoch()

    print("Training process has finished")

  def fitEpoch(self):
    currentLoss = 0.0

    for i, data in enumerate(self.data):
      # Get input aand its corresponding groundtruth output
      inputs, target = data

      self.optimiser.zero_grad()

      # get output from the model, given the inputs
      outputs = self.model(inputs)

      # get loss for the predicted output
      loss = self.model.loss(outputs, target)

      # get gradients w.r.t the parameters of the model
      loss.backward()

      # update the parameters (perform optimization)
      self.optimiser.step()

      currentLoss += loss.item()
      if i % 10 == 9:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, currentLoss / 10))
          currentLoss = 0.0

In [6]:
trainSet = DataLoader().load(dataset="/train", batchSize=10, shuffle=True, workers=2)
model = MLP(lr=1e-04)
trainer = Trainer(nEpochs=1)
trainer.fit(model, trainSet)

KeyError: 124